# 설치

In [7]:
!pip install qdrant-client sentence-transformers

In [11]:
!pip install qdrant-client

In [14]:
!pip install python-dotenv

# 1. qdrant에 데이터 벡터화하여 저장하기

In [16]:
from dotenv import load_dotenv
import os
load_dotenv()

# 환경변수 가져오기
qdrant_api_key = os.getenv("QDRANT_API_KEY")
qdrant_url = os.getenv("QDRANT_URL")

In [17]:
from qdrant_client import QdrantClient

client = QdrantClient(
    url=qdrant_url,
    api_key=qdrant_api_key
)

print(client.get_collections())

collections=[]


In [18]:
import pandas as pd
import numpy as np

path = '21대국정조사.csv'
data = pd.read_csv(path)

print(data.head())

    회의번호          발언자                                               발언내용
0  52512  위원장직무대행 박성민  의석을 정돈해 주시기 바랍니다.  국회법 제52조 및 제3호에 따라 특별위원회 재적...
1  52512  위원장직무대행 박성민  현재 안건이 미정인 상태로 회의가 열리게 되었기 때문에 추가로 안건을 상정하여 회의...
2  52512  위원장직무대행 박성민  의사일정 제1항 위원장 선임의 건을 상정합니다.   특별위원회 위원장은 국회법 제4...
3  52512       조응천 위원                         우상호 위원을 추천합니다.            
4  52512  위원장직무대행 박성민  또 추천하실 위원님 안 계십니까?    (「없습니다」 하는 위원 있음)  우상호 위...


In [19]:
from sentence_transformers import SentenceTransformer

# 벡터화 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')

# 벡터화
data['vector'] = data['발언내용'].apply(lambda x: model.encode(x))
print("벡터화 완료")

c:\Users\user\.conda\envs\malcam\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\user\.conda\envs\malcam\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


벡터화 완료


In [5]:
# 벡터화 후 파일로 저장
data.to_parquet("벡터화_데이터.parquet")  # 벡터화된 데이터를 저장

In [23]:
# 컬렉션 생성
client.recreate_collection(
    collection_name="meeting_records",
    vectors_config={"size": 384, "distance": "Cosine"}  # 벡터 크기 및 거리 계산 방식
)
print("컬렉션 생성 완료")

C:\Users\user\AppData\Local\Temp\ipykernel_13160\1063533258.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


컬렉션 생성 완료


In [24]:
points = [
    {
        "id": idx,
        "vector": row['vector'],
        "payload": {
            "발언자": row['발언자'],
            "회의번호": row['회의번호'],
            "발언내용": row['발언내용']
        }
    }
    for idx, row in data.iterrows()
]

# 데이터 삽입
batch_size = 20  # 배치 크기 설정
for i in range(0, len(points), batch_size):
    batch = points[i:i + batch_size]
    client.upsert(
        collection_name="meeting_records",
        points=batch
    )
    print(f"배치 {i // batch_size + 1} 삽입 완료")

# 삽입된 데이터 개수 확인
collection_info = client.get_collection("meeting_records")
print(f"총 데이터 개수: {collection_info.vectors_count}")

배치 1 삽입 완료
배치 2 삽입 완료
배치 3 삽입 완료
배치 4 삽입 완료
배치 5 삽입 완료
배치 6 삽입 완료
배치 7 삽입 완료
배치 8 삽입 완료
배치 9 삽입 완료
배치 10 삽입 완료
배치 11 삽입 완료
배치 12 삽입 완료
배치 13 삽입 완료
배치 14 삽입 완료
배치 15 삽입 완료
배치 16 삽입 완료
배치 17 삽입 완료
배치 18 삽입 완료
배치 19 삽입 완료
배치 20 삽입 완료
배치 21 삽입 완료
배치 22 삽입 완료
배치 23 삽입 완료
배치 24 삽입 완료
배치 25 삽입 완료
배치 26 삽입 완료
배치 27 삽입 완료
배치 28 삽입 완료
배치 29 삽입 완료
배치 30 삽입 완료
배치 31 삽입 완료
배치 32 삽입 완료
배치 33 삽입 완료
배치 34 삽입 완료
배치 35 삽입 완료
배치 36 삽입 완료
배치 37 삽입 완료
배치 38 삽입 완료
배치 39 삽입 완료
배치 40 삽입 완료
배치 41 삽입 완료
배치 42 삽입 완료
배치 43 삽입 완료
배치 44 삽입 완료
배치 45 삽입 완료
배치 46 삽입 완료
배치 47 삽입 완료
배치 48 삽입 완료
배치 49 삽입 완료
배치 50 삽입 완료
배치 51 삽입 완료
배치 52 삽입 완료
배치 53 삽입 완료
배치 54 삽입 완료
배치 55 삽입 완료
배치 56 삽입 완료
배치 57 삽입 완료
배치 58 삽입 완료
배치 59 삽입 완료
배치 60 삽입 완료
배치 61 삽입 완료
배치 62 삽입 완료
배치 63 삽입 완료
배치 64 삽입 완료
배치 65 삽입 완료
배치 66 삽입 완료
배치 67 삽입 완료
배치 68 삽입 완료
배치 69 삽입 완료
배치 70 삽입 완료
배치 71 삽입 완료
배치 72 삽입 완료
배치 73 삽입 완료
배치 74 삽입 완료
배치 75 삽입 완료
배치 76 삽입 완료
배치 77 삽입 완료
배치 78 삽입 완료
배치 79 삽입 완료
배치 80 삽입 완료
배치 81 삽입 완료
배치 82 삽입 완료
배치 83 삽입 완료
배치 84 삽입 완료
배

In [25]:
# 삽입된 데이터 개수 확인
collection_info = client.get_collection("meeting_records")
print(f"총 데이터 개수: {collection_info.vectors_count}")

총 데이터 개수: None


In [26]:
results = client.scroll(
    collection_name="meeting_records",
    offset=None,  # 처음부터 검색
    limit=5  # 상위 5개 결과 반환
)

# 검색 결과 출력
for result in results[0]:  # scroll 반환값은 (points, next_page_offset)의 튜플
    print("ID:", result.id)
    print("발언내용:", result.payload['발언내용'])
    print("회의번호:", result.payload['회의번호'])
    print("발언자:", result.payload['발언자'])
    print()

ID: 0
발언내용: 의석을 정돈해 주시기 바랍니다.  국회법 제52조 및 제3호에 따라 특별위원회 재적위원 4분의 1 이상의 회의 개회 요구가 있었습니다. 이에 제400회 국회(정기회) 제1차 용산 이태원 참사 진상규명과 재발방지를 위한 국정조사특별위원회를 개회하겠습니다.  국회법 제47조제2항에 따라 위원장 선임 시까지 위원장 직무를 대행하게 되었습니다.            
회의번호: 52512
발언자: 위원장직무대행 박성민

ID: 1
발언내용: 현재 안건이 미정인 상태로 회의가 열리게 되었기 때문에 추가로 안건을 상정하여 회의를 진행하고자 합니다.   위원님 여러분 이의가 없습니까?     (「예」 하는 위원 있음)  이의가 없으시면 의사일정을 변경하여 안건을 상정하도록 하겠습니다.            
회의번호: 52512
발언자: 위원장직무대행 박성민

ID: 2
발언내용: 의사일정 제1항 위원장 선임의 건을 상정합니다.   특별위원회 위원장은 국회법 제47조제1항에 따라 특별위원회에서 호선하고 본회의에 보고하도록 되어 있습니다.   위원장 선임 방법은 위원님들께서 특별한 의견이 없으시면 그간의 특별위원회 관례에 따라 구두 추천에 의하여 선임하고자 합니다.  위원님 여러분 이의가 없으십니까?    (「예」 하는 위원 있음)  그러면 위원님들께서 이번 국정조사를 원만하고 공정하게 이끌어 주실 가장 적임자라고 생각하시는 위원님을 우리 위원회 위원장으로 추천해 주시기 바랍니다.  조응천 위원님.            
회의번호: 52512
발언자: 위원장직무대행 박성민

ID: 3
발언내용: 우상호 위원을 추천합니다.            
회의번호: 52512
발언자: 조응천 위원

ID: 4
발언내용: 또 추천하실 위원님 안 계십니까?    (「없습니다」 하는 위원 있음)  우상호 위원님을 위원장으로 선임하고자 합니다.  위원님 여러분 이의 없으십니까?    (「예」 하는 위원 있음)  이의 없으므로 가결되었음을 선포합니다.  협조해 주신 위원님들께 감

# 2. 벡터 db 호출하여 발언내용 요약하기

In [32]:
import random
from transformers import pipeline

# Qdrant에서 모든 데이터 불러오기
def fetch_all_data_from_qdrant(collection_name, batch_size=50):
    all_results = []
    offset = None

    while True:
        results, next_page_offset = client.scroll(
            collection_name=collection_name,
            offset=offset,
            limit=batch_size
        )
        all_results.extend(results)

        if not next_page_offset:
            break

        offset = next_page_offset

    return all_results

# 무작위 샘플링
def random_sample(data, sample_size=100):
    return random.sample(data, sample_size)

In [35]:
# BART 요약 모델 로드
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# 텍스트 요약 함수
def summarize_text(text):
    try:
        summary = summarizer(text, max_length=150, min_length=50, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        print(f"요약 실패: {e}")
        return "요약 실패"

# 샘플 데이터 요약 실행
def summarize_random_sample(collection_name, sample_size=100):
    # Qdrant에서 모든 데이터 가져오기
    all_data = fetch_all_data_from_qdrant(collection_name)
    print(f"총 데이터 개수: {len(all_data)}")

    # 무작위 샘플링
    sampled_data = random_sample(all_data, sample_size)
    print(f"무작위로 {sample_size}개의 데이터를 샘플링했습니다.")

    # 요약 실행
    summarized_data = []
    for item in sampled_data:
        original_text = item.payload['발언내용']
        summarized_text = summarize_text(original_text)
        summarized_data.append({
            "ID": item.id,
            "원문": original_text,
            "요약": summarized_text
        })
        
    # 결과 저장
    df = pd.DataFrame(summarized_data)
    output_path = "qdrant_요약_샘플_결과.csv"
    df.to_csv(output_path, index=False, encoding="utf-8-sig")

    print(f"요약된 샘플 데이터가 '{output_path}'에 저장되었습니다.")

# 실행
summarize_random_sample("meeting_records", sample_size=100)

총 데이터 개수: 9682
무작위로 100개의 데이터를 샘플링했습니다.
요약 실패: index out of range in self


Your max_length is set to 150, but your input_length is only 47. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)
Your max_length is set to 150, but your input_length is only 33. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 150, but your input_length is only 63. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=31)
Your max_length is set to 150, but your input_length is only 18. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
Your 

요약 실패: index out of range in self


Your max_length is set to 150, but your input_length is only 18. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
Your max_length is set to 150, but your input_length is only 49. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)
Your max_length is set to 150, but your input_length is only 84. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 150, but your input_length is only 82. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)
Your 

요약 실패: index out of range in self


...

요약 실패: index out of range in self


Your max_length is set to 150, but your input_length is only 27. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length is set to 150, but your input_length is only 68. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)
Your max_length is set to 150, but your input_length is only 27. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length is set to 150, but your input_length is only 89. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)
Your

요약 실패: index out of range in self


Your max_length is set to 150, but your input_length is only 33. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 150, but your input_length is only 112. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=56)
Your max_length is set to 150, but your input_length is only 71. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=35)
Your max_length is set to 150, but your input_length is only 33. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
You

요약된 샘플 데이터가 'qdrant_요약_샘플_결과.csv'에 저장되었습니다.


In [36]:
sameple = pd.read_csv('qdrant_요약_샘플_결과.csv')
sameple.head()


,ID,원문,요약
0,5914,"저는 진짜 이해할 수가 없는데 그러면 결국은, 용산소방서장의 인파 통제 요청이 23...",요약 실패
1,2347,"아니, 우리가 현장조사 나갔을 때는 지금 말씀하신 취지로 보고 하셨잖아요. 그러니까...","‘ ‘”’ ‘’, “” ””, “ ’’ “, ”, ’”. “.” “ ..."
2,4513,김광호 증인 말씀하십시오.,"김광호 증인 “십’s” ‘‘’ “’’ ”“” ’” ”, “ ’,’..."
3,2024,"예, 그렇습니다.","“I’m sorry, I’ve got to go.” “What’s w..."
4,4414,"이것은 진짜 너무들 해요, 정말 너무들 해.","넉 ‘’ ‘”’ “’’, ’”, ‘, ’, ”, “,” ..."


# 3. gptapi를 이용하여 요약

In [27]:
all_data = fetch_all_data_from_qdrant("meeting_records")
print(f"불러온 데이터 개수: {len(all_data)}")

불러온 데이터 개수: 9682


In [28]:
for item in all_data[:5]:  # 샘플 데이터 5개만 출력
    print(item.payload)

{'발언자': '위원장직무대행 박성민', '회의번호': 52512, '발언내용': '의석을 정돈해 주시기 바랍니다.  국회법 제52조 및 제3호에 따라 특별위원회 재적위원 4분의 1 이상의 회의 개회 요구가 있었습니다. 이에 제400회 국회(정기회) 제1차 용산 이태원 참사 진상규명과 재발방지를 위한 국정조사특별위원회를 개회하겠습니다.  국회법 제47조제2항에 따라 위원장 선임 시까지 위원장 직무를 대행하게 되었습니다.            '}
{'발언자': '위원장직무대행 박성민', '회의번호': 52512, '발언내용': '현재 안건이 미정인 상태로 회의가 열리게 되었기 때문에 추가로 안건을 상정하여 회의를 진행하고자 합니다.   위원님 여러분 이의가 없습니까?     (「예」 하는 위원 있음)  이의가 없으시면 의사일정을 변경하여 안건을 상정하도록 하겠습니다.            '}
{'발언자': '위원장직무대행 박성민', '회의번호': 52512, '발언내용': '의사일정 제1항 위원장 선임의 건을 상정합니다.   특별위원회 위원장은 국회법 제47조제1항에 따라 특별위원회에서 호선하고 본회의에 보고하도록 되어 있습니다.   위원장 선임 방법은 위원님들께서 특별한 의견이 없으시면 그간의 특별위원회 관례에 따라 구두 추천에 의하여 선임하고자 합니다.  위원님 여러분 이의가 없으십니까?    (「예」 하는 위원 있음)  그러면 위원님들께서 이번 국정조사를 원만하고 공정하게 이끌어 주실 가장 적임자라고 생각하시는 위원님을 우리 위원회 위원장으로 추천해 주시기 바랍니다.  조응천 위원님.            '}
{'발언자': '조응천 위원', '회의번호': 52512, '발언내용': '우상호 위원을 추천합니다.            '}
{'발언자': '위원장직무대행 박성민', '회의번호': 52512, '발언내용': '또 추천하실 위원님 안 계십니까?    (「없습니다」 하는 위원 있음)  우상호 위원님을 위원장으로 선임하고자 합니다.  위원님 여러분 이의 

In [76]:
import openai
from qdrant_client import QdrantClient
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

# 환경 변수 로드 및 OpenAI API 설정
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

# Qdrant 클라이언트 초기화
qdrant_api_key = os.getenv("QDRANT_API_KEY")
qdrant_url = os.getenv("QDRANT_URL")

# 1. Qdrant에서 데이터 불러오기
def fetch_all_data_from_qdrant(collection_name, batch_size=50):
    all_results = []
    offset = None
    while True:
        results, next_page_offset = client.scroll(
            collection_name=collection_name,
            offset=offset,
            limit=batch_size
        )
        all_results.extend(results)
        if not next_page_offset:
            break
        offset = next_page_offset
    return all_results

In [72]:
# 2. 회의번호별로 발언내용 합치기
def group_by_meeting(data):
    grouped_data = {}
    for item in data:
        meeting_number = item.payload.get('회의번호')
        speech = item.payload.get('발언내용')
        if not meeting_number or not speech:
            continue  # 회의번호나 발언내용이 없는 항목 건너뛰기
        if meeting_number not in grouped_data:
            grouped_data[meeting_number] = []
        grouped_data[meeting_number].append(speech)

    # 발언내용을 회의번호별로 합치기
    grouped_data = {key: " ".join(value) for key, value in grouped_data.items()}
    return grouped_data

In [ ]:
# 3. GPT-4 Turbo 요약 함수
def gpt4_turbo_summarize(text):
    """
    GPT-4 Turbo를 사용하여 텍스트를 요약합니다.
    """
    try:
        response = openai.Completion.create(
            engine="gpt-4-turbo",
            prompt=f"다음 텍스트를 요약해 주세요:\n{text}",
            max_tokens=150,  # 요약 텍스트 최대 길이
            temperature=0.5
        )
        return response.choices[0].text.strip()
    except Exception as e:
        print(f"요약 실패: {e}")
        return "요약 실패"

In [74]:
# 4. 그룹화된 데이터를 요약
def summarize_grouped_data(grouped_data):
    """
    합쳐진 발언내용을 GPT-4 Turbo로 요약합니다.
    """
    summarized_data = []
    for meeting_number, content in grouped_data.items():
        print(f"회의번호 {meeting_number} 요약 중...")
        summarized_text = gpt4_turbo_summarize(content)
        summarized_data.append({
            "회의번호": meeting_number,
            "원문": content,
            "요약": summarized_text
        })
    return summarized_data

In [75]:
# 5. 전체 프로세스
def summarize_by_meeting(collection_name="meeting_records"):
    """
    Qdrant 데이터를 회의번호별로 그룹화하고 요약하는 전체 프로세스.
    """
    # Qdrant 데이터 불러오기
    print("Qdrant 데이터 불러오는 중...")
    all_data = fetch_all_data_from_qdrant(collection_name)
    print(f"총 데이터 개수: {len(all_data)}")

    # 회의번호별로 그룹화
    print("회의번호별로 데이터를 그룹화 중...")
    grouped_data = group_by_meeting(all_data)
    print(f"총 회의번호 개수: {len(grouped_data)}")

    # 그룹화된 데이터 요약
    print("그룹화된 데이터를 요약 중...")
    summarized_data = summarize_grouped_data(grouped_data)

    # 결과 저장
    print("결과를 저장 중...")
    df = pd.DataFrame(summarized_data)
    output_path = "qdrant_회의별_요약_결과.csv"
    df.to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"요약된 데이터가 '{output_path}'에 저장되었습니다.")